<a href="https://colab.research.google.com/github/Farah-Abdirahman/sample-adk-agent/blob/main/ADK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install google-adk -q
# !pip install litellm -q

In [ ]:
from google.colab import userdata
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")


Libraries imported.


In [ ]:
import os
import google.generativeai as genai # Import the generativeai library
from google.colab import userdata
# Get Key from secrets
google_api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = google_api_key
# print(google_api_key)
# # Configure the generativeai library with the API key
# genai.configure(api_key=google_api_key)
# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"



#**Build Your First Intelligent Agent Team: A Progressive Kabarak Uni Bot with ADK**
The Kabarak Campus Assistant (KCA) is a simple AI agent built using a Large Language Model (LLM) that provides helpful campus-related information for students, staff, and visitors at Kabarak University

**🎯 Objective Create a basic AI agent that:**

Understands user questions about Kabarak campus (e.g., weather, events, library hours).

Uses a tool to retrieve specific information.

Responds clearly using an LLM like Gemini Flash.

**Let's Take a look**

In the Agent Developer Kit (ADK), tools are Python functions that give your agent practical abilities—like retrieving data, calling APIs, or performing tasks.

We’ll start with a mock campus info tool, returning sample answers without needing real APIs (easy to extend later).

**NB**: **📌 Why the docstring matters:
This helps the LLM understand:

What the tool is for

When to use it

The input (topic: str)

The structure of the response (dict with keys like status, report, etc.)**

In [ ]:
# @title Define the get_kabarak_info Tool
def get_kabarak_info(topic: str) -> dict:
    """Retrieves information about Kabarak University based on a specific topic.

    Args:
        topic (str): The topic of interest (e.g., "weather", "events", "library").

    Returns:
        dict: A dictionary containing campus-related information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with relevant details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_kabarak_info called for topic: {topic} ---")  # Log tool execution
    topic_normalized = topic.lower().replace(" ", "")

    # Mock data for Kabarak campus
    kabarak_mock_db = {
        "weather": {
            "status": "success",
            "report": "The weather at Kabarak Main Campus is sunny with mild winds, around 23°C."
        },
        "events": {
            "status": "success",
            "report": "Today's events: Career Fair at the Auditorium (10am–4pm), and Chapel Service at 8am."
        },
        "library": {
            "status": "success",
            "report": "The university library is open from 8am to 9pm. Online resources are available via the student portal."
        },
        "cafeteria": {
            "status": "success",
            "report": "The cafeteria is serving local dishes and snacks today. Lunch runs from 12:00pm to 2:00pm."
        }
    }

    if topic_normalized in kabarak_mock_db:
        return kabarak_mock_db[topic_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have information about '{topic}'."}

# 🔍 Example test
print(get_kabarak_info("weather"))
print(get_kabarak_info("events"))
print(get_kabarak_info("cafeteria"))
print(get_kabarak_info("sports"))


--- Tool: get_kabarak_info called for topic: weather ---
{'status': 'success', 'report': 'The weather at Kabarak Main Campus is sunny with mild winds, around 23°C.'}
--- Tool: get_kabarak_info called for topic: events ---
{'status': 'success', 'report': "Today's events: Career Fair at the Auditorium (10am–4pm), and Chapel Service at 8am."}
--- Tool: get_kabarak_info called for topic: cafeteria ---
{'status': 'success', 'report': 'The cafeteria is serving local dishes and snacks today. Lunch runs from 12:00pm to 2:00pm.'}
--- Tool: get_kabarak_info called for topic: sports ---
{'status': 'error', 'error_message': "Sorry, I don't have information about 'sports'."}


In [ ]:
# @title Define the Kabarak Campus Agent

# Use the same or a different Gemini model as needed
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH  # or use a string if using a local version

# Define the agent
kabarak_agent = Agent(
    name="kabarak_agent_v1",
    model=AGENT_MODEL,
    description="Provides helpful information about Kabarak University.",
    instruction=(
        "You are a helpful assistant for Kabarak University students, staff, and visitors. "
        "When asked about campus-related topics like weather, events, the library, or cafeteria, "
        "use the 'get_kabarak_info' tool to retrieve the information. "
        "If the tool returns an error, respond politely and suggest valid topics. "
        "If successful, present the report in a clear and friendly tone."
    ),
    tools=[get_kabarak_info],  # Attach the campus info tool
)

print(f"Agent '{kabarak_agent.name}' created using model '{AGENT_MODEL}'.")


Agent 'kabarak_agent_v1' created using model 'gemini-2.0-flash'.


In [ ]:
# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "kabarak_info_agent"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=kabarak_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='kabarak_info_agent', User='user_1', Session='session_001'
Runner created for agent 'kabarak_agent_v1'.


In [ ]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [ ]:
# @title Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("What is the weather like at Kabarak?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    await call_agent_async("Tell me about the library hours.",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    await call_agent_async("Is there a football match today?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()

# --- OR ---

# Uncomment the following lines if running as a standard Python script (.py file):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"An error occurred: {e}")


>>> User Query: What is the weather like at Kabarak?


--- Tool: get_kabarak_info called for topic: weather ---
<<< Agent Response: Hello! The weather at Kabarak Main Campus is sunny with mild winds, around 23°C.


>>> User Query: Tell me about the library hours.


--- Tool: get_kabarak_info called for topic: library ---
<<< Agent Response: The university library is open from 8am to 9pm. Online resources are available via the student portal.


>>> User Query: Is there a football match today?


--- Tool: get_kabarak_info called for topic: events ---
<<< Agent Response: Today's events include a Career Fair at the Auditorium (10am–4pm), and Chapel Service at 8am. There is no mention of a football match.



# Building an Agent Team – Delegation for Greetings, Farewells & Campus Info
In Steps 1 and 2, we created a single-purpose agent for Kabarak University that provided weather and campus-related information.

But real users might also say things like:

“Hi there!”

“Thanks, goodbye!”

“What's the weather like today?”

Instead of crowding one agent with everything, let’s make a smarter, modular agent team.

**Define the Tools**


In [ ]:
def say_hello() -> dict:
    """Responds to user greetings."""
    return {"status": "success", "message": "Hello! Welcome to Kabarak University Assistant. How can I help you today?"}

def say_goodbye() -> dict:
    """Responds to user farewells."""
    return {"status": "success", "message": "Goodbye! Have a great day at Kabarak University."}


In [ ]:
greeting_agent = Agent(
    name="greeting_agent",
    model=MODEL_GEMINI_2_0_FLASH,  # Lightweight model is fine
    description="Handles user greetings.",
    instruction="Respond warmly to any greetings like 'hello', 'hi', or 'good morning'. Use the 'say_hello' tool.",
    tools=[say_hello]
)


In [ ]:
farewell_agent = Agent(
    name="farewell_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Handles user goodbyes and sign-offs.",
    instruction="Respond politely to farewells like 'bye', 'see you later', or 'goodnight'. Use the 'say_goodbye' tool.",
    tools=[say_goodbye]
)


**Update the Kabarak Agent to Act as the Root Agent**

In [ ]:
# @title Define Greeting and Farewell Sub-Agents

# --- Greeting Agent ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model = MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                    "Use the 'say_hello' tool to generate the greeting. "
                    "If the user provides their name, make sure to pass it to the tool. "
                    "Do not engage in any other conversation or tasks.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' created using model '{greeting_agent.model}'.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Check API Key ({greeting_agent.model}). Error: {e}")

# --- Farewell Agent ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model = MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                    "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                    "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                    "Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' created using model '{farewell_agent.model}'.")
except Exception as e:
    print(f"❌ Could not create Farewell agent. Check API Key ({farewell_agent.model}). Error: {e}")


✅ Agent 'greeting_agent' created using model 'gemini-2.0-flash'.
✅ Agent 'farewell_agent' created using model 'gemini-2.0-flash'.


t

In [ ]:
# @title Define the Root Agent with Sub-Agents (Kabarak Agent Team)

root_agent = None
runner_root = None  # Initialize runner variable

# Ensure all sub-agents and the Kabarak info tool are ready
if greeting_agent and farewell_agent and 'get_kabarak_info' in globals():

    # Use a strong yet cost-effective model for coordination
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    kabarak_agent_team = Agent(
        name="kabarak_agent_v2",
        model=root_agent_model,
        description=(
            "The main coordinator agent for Kabarak University information. "
            "Answers university-specific questions and delegates greetings/farewells."
        ),
        instruction=(
            "You are the Kabarak University Info Agent coordinating a team. Your primary responsibility is to provide accurate information about Kabarak University. "
            "Use the 'get_kabarak_info' tool ONLY for queries related to university details (e.g., courses, campus facilities, admissions, contact info). "
            "You have specialized sub-agents: "
            "1. 'greeting_agent': Handles greetings like 'Hello', 'Hi'. Delegate to it for these. "
            "2. 'farewell_agent': Handles farewells like 'Bye', 'See you'. Delegate to it for these. "
            "Analyze the user's input. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
            "If it's a Kabarak-specific question, respond using 'get_kabarak_info'. "
            "For unrelated queries, respond politely that you can only provide Kabarak-related info."
        ),
        tools=[get_kabarak_info],  # Core tool for Kabarak info
        sub_agents=[greeting_agent, farewell_agent]  # Link greeting and farewell specialists
    )

    print(f"✅ Root Agent '{kabarak_agent_team.name}' created using model '{root_agent_model}' with sub-agents: {[sa.name for sa in kabarak_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed or 'get_kabarak_info' tool is missing.")
    if not greeting_agent: print(" - Greeting Agent is missing.")
    if not farewell_agent: print(" - Farewell Agent is missing.")
    if 'get_kabarak_info' not in globals(): print(" - get_kabarak_info function is missing.")


✅ Root Agent 'kabarak_agent_v2' created using model 'gemini-2.0-flash' with sub-agents: ['greeting_agent', 'farewell_agent']


In [ ]:
# @title Interact with the Agent Team (Kabarak University Version)
import asyncio  # Ensure asyncio is imported

# Determine which root agent variable is defined
root_agent_var_name = 'root_agent'  # Default name from earlier steps

if 'kabarak_agent_team' in globals():  # Check for your Kabarak agent team name
    root_agent_var_name = 'kabarak_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'kabarak_agent_team') not found. Cannot define run_team_conversation.")
    root_agent = None  # Prevent NameError if used later

# Only define and run conversation if root agent exists
if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- Testing Kabarak Agent Team Delegation ---")
        session_service = InMemorySessionService()
        APP_NAME = "kabarak_agent_team_app"
        USER_ID = "user_1_kabarak_team"
        SESSION_ID = "session_001_kabarak_team"

        session = session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner(  # Or use InMemoryRunner if preferred
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"Runner created for agent '{actual_root_agent.name}'.")

        # Test greeting delegation
        await call_agent_async(
            query="Hello Kabarak!",
            runner=runner_agent_team,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        # Test Kabarak info request delegation (replace with relevant query)
        await call_agent_async(
            query="Tell me about the library hours?",
            runner=runner_agent_team,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        # Test farewell delegation
        await call_agent_async(
            query="Thanks, goodbye!",
            runner=runner_agent_team,
            user_id=USER_ID,
            session_id=SESSION_ID
        )

    # Execute the async conversation runner
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_team_conversation()

    # If you are running this as a standard Python script (.py),
    # comment out the above line and uncomment below:

    """
    import asyncio
    if __name__ == "__main__":
        print("Executing using 'asyncio.run()' for standard Python script...")
        try:
            asyncio.run(run_team_conversation())
        except Exception as e:
            print(f"Error during async execution: {e}")
    """
else:
    print("\n⚠️ Skipping Kabarak agent team conversation execution because the root agent was not found.")


Attempting execution using 'await' (default for notebooks)...

--- Testing Kabarak Agent Team Delegation ---
Session created: App='kabarak_agent_team_app', User='user_1_kabarak_team', Session='session_001_kabarak_team'
Runner created for agent 'kabarak_agent_v2'.

>>> User Query: Hello Kabarak!


<<< Agent Response: Hello! Welcome to Kabarak University Assistant. How can I help you today?


>>> User Query: Tell me about the library hours?


--- Tool: get_kabarak_info called for topic: library ---
<<< Agent Response: The university library is open from 8am to 9pm. Online resources are available via the student portal.


>>> User Query: Thanks, goodbye!


<<< Agent Response: Goodbye! Have a great day at Kabarak University.

